In [1]:
import pygame

####################################################################
# 기본 초기화(반드시 해야 하는 것들)
pygame.init()

# 화면 크기 설정
screen_width = 480 # 가로
screen_height = 640 # 세로
screen = pygame.display.set_mode((screen_width,screen_height))


# 화면 타이틀 설정
pygame.display.set_caption("로니콜먼의 모험")

# FPS(frame per second)
clock = pygame.time.Clock()
#####################################################################

# 1. 사용자 게임 초기화 (배경화면, 캐릭터, 게임 이미지, 좌표, 속도, 폰트 등)

# 배경 만들기
background = pygame.image.load(r"C:\python\project\game\pygame_project\images\background1.png")

# 스테이지 만들기
stage = pygame.image.load(r"C:\python\project\game\pygame_project\images\stage.png")
stage_size = stage.get_rect().size
stage_height = stage_size[1] # 스테이지 높이 위에 캐릭터를 두기 위함 # get_rect().size를 하면 이미지 크기 만큼을 (n,m)형식으로 가져옴 # [n]은 n-1번째 항목 호출


# 캐릭터 만들기
character = pygame.image.load(r"C:\python\project\game\pygame_project\images\character.png")
character_size = character.get_rect().size
character_width = character_size[0]
character_height =  character_size[1]
character_xpos = (screen_width/2) - (character_width/2)
character_ypos = (screen_height - stage_height) - character_height

# 캐릭터 이동방향
character_to_x = 0
character_to_y = 0

# 캐릭터 이동속도
character_speed = 5

# 무기 만들기
weapon = pygame.image.load(r"C:\python\project\game\pygame_project\images\weapon.png")
weapon_size = weapon.get_rect().size
weapon_width = weapon_size[0]

# 무기는 한 번에 여려 발 발사 가능
weapons = []

# 무기 이동 속도
weapon_speed = 10

# 공 만들기 (4개 크기에 대해 따로 처리)
ball_images = [
    pygame.image.load(r"C:\python\project\game\pygame_project\images\ballon1.png"),
    pygame.image.load(r"C:\python\project\game\pygame_project\images\ballon2.png"),
    pygame.image.load(r"C:\python\project\game\pygame_project\images\ballon3.png"),
    pygame.image.load(r"C:\python\project\game\pygame_project\images\ballon4.png")
    ]

# 공 크기에 따른 최초 스피드
ball_speed_y = [-18,-15,-12,-9] # index 0,1,2,3에 해당하는 값

# 공들
balls = []

# 최초 발생하는 큰 공 추가
balls.append({
    "pos_x" : 50, # 공의 x좌표
    "pos_y" : 50, # 공의 y좌표
    "img_idx" : 0,  # 공의 이미지 인덱스
    "to_x" : 3, # x축 이동방향, -3 이면 왼쪽으로, +3 이면 오른쪽으로
    "to_y" : -6, # y축 이동방향, -6 이면 위로
    "init_spd_y" : ball_speed_y[0] # y 최초 속도
})


running = True
while running: 
    dt = clock.tick(60)
    
    # 2. 이벤트 처리(키보드, 마우스 등)
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            running = False 
            
        if event.type == pygame.KEYDOWN: # 키가 눌러졌는지 확인
            if event.key == pygame.K_RIGHT: # 캐릭터를 오른쪽으로
                character_to_x += character_speed
            elif event.key == pygame.K_LEFT: # 캐릭터를 왼쪽으로
                character_to_x -= character_speed
            elif event.key == pygame.K_SPACE:
                weapon_xpos = character_xpos + (character_width/2) - (weapon_width/2)
                weapon_ypos = character_ypos 
                weapons.append([weapon_xpos, weapon_ypos])
                
        if event.type == pygame.KEYUP: # 방향키를 떼면 멈춘다
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                character_to_x = 0
            elif event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                character_to_y = 0   
                
    # 3. 게임 캐릭터 위치 정의
    if character_xpos < 0:
        character_xpos = 0
    elif character_xpos > screen_width - character_width:
        character_xpos = screen_width - character_width
            
    character_xpos += character_to_x
    
    # 무기 위치 조정
    weapons = [[ w[0], (w[1]-weapon_speed) ] for w in weapons] # for 구문으로 인해 무한반복(weapons는 계속 변하기 때문)
    # w가 할당받은 리스트 묶음을 앞에[]에 각각 할당
    # w[0]이 w의 0번째를, w[1]이 1번째를 할당받고 이것이 weapons내 리스트 값 전체에 반복
    # weapon_speed를 빼주는 이유는 위로 올라가는 것이 y좌표에서는 감소하는 것이기 때문
    
    # 천장에 닿은 무기 없애기
    weapons = [[ w[0], w[1]] for w in weapons if w[1] > 0] # y좌표가 0보다 큰 값만 할당 # 복습 필요
    
    # 공 위치 정의
    for ball_idx, ball_value in enumerate(balls):
        ball_xpos = ball_value["pos_x"]
        ball_ypos = ball_value["pos_y"]
        ball_img_idx = ball_value["img_idx"]
        
        ball_size = ball_images[ball_img_idx].get_rect().size
        ball_width = ball_size[0]
        ball_height = ball_size[1]
        
        # 가로 벽에 닿았을 때 공의 이동 방향 변경(튕겨지는 효과)
        if ball_xpos < 0 or ball_xpos > screen_width - ball_width:
            ball_value["to_x"] = ball_value["to_x"] * -1
            
        # 세로로 스테이지에 튕겨서 올라가는 처리
        if ball_ypos >= screen_height - stage_height - ball_height: # y 좌표가 크다는 것은 더 아래에 있는 것임
            ball_value["to_y"] = ball_value["init_spd_y"]
        else: # 그 외의 모든 경우에는 속도를 증가
            ball_value["to_y"] += 0.5
            
        ball_value["pos_x"] += ball_value["to_x"]
        ball_value["pos_y"] += ball_value["to_y"]
    
    # 4. 충돌 처리
  
    
    
    # 5. 화면에 그리기
    screen.blit(background, (0,0)) # 표시할 객체와 표시할 위치
    
    for weapon_xpos, weapon_ypos in weapons:
        screen.blit(weapon,(weapon_xpos,weapon_ypos)) # 무기가 캐릭터, 스테이지를 덮으면 안되기 때문에 순서를 앞으로 둠
    
    for idx, val in enumerate(balls):
        ball_xpos = val["pos_x"]
        ball_ypos = val["pos_y"]
        ball_img_idx = val["img_idx"]
        screen.blit(ball_images[ball_img_idx],(ball_xpos,ball_ypos))
        
    screen.blit(stage, (0,screen_height - stage_height))
    screen.blit(character,(character_xpos,character_ypos))
    
    
    pygame.display.update() # 게임화면 다시 그리기!(반드시) #blit 아래에 있어야 한다
    

# pygame 종료
pygame.quit()

pygame 2.5.0 (SDL 2.28.0, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
